# Семинар 4. Линейные модели в задачах классификации

Подключение библиотек. 
$
\newcommand{\R}{\mathbb{R}}
\newcommand{\X}{\mathbb{X}}
\newcommand{\norm}[1]{\lVert #1 \rVert}
\newcommand{\abs}[1]{\left| #1 \right|}
\newcommand{\E}{\mathbb{E}}
\newcommand{\D}{\mathbb{D}}
\renewcommand{\Prob}{\mathbb{P}}
\renewcommand{\le}{\leqslant}
\renewcommand{\ge}{\geqslant}
\newcommand{\eps}{\varepsilon}
\newcommand{\Normal}{\mathcal{N}}
$

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as wg

from matplotlib.colors import ListedColormap

RdGn = ListedColormap(['red', 'green'])
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
from sklearn.datasets import make_classification, make_circles
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# вспомогательная функция
def plot_classification(X, y, model, show_line=False, title='', X_transform=None):
    X_pred = X_transform(X) if X_transform is not None else X
    pred = model.predict(X_pred)
    
    idx = (y == pred) & (y == 1)
    plt.scatter(X[idx, 0], X[idx, 1], color='green', label='true=1, pred=1')

    idx = (y != pred) & (y == 1)
    plt.scatter(X[idx, 0], X[idx, 1], color='blue', label='true=1, pred=0')

    idx = (y == pred) & (y == 0)
    plt.scatter(X[idx, 0], X[idx, 1], color='red', label='true=0, pred=0')

    idx = (y != pred) & (y == 0)
    plt.scatter(X[idx, 0], X[idx, 1], color='yellow', label='true=0, pred=1')

    X1, X2 = X[:, 0], X[:, 1]
    X1 = np.linspace(X1.min(), X1.max(), 128)
    X2 = np.linspace(X2.min(), X2.max(), 128)
    X1, X2 = np.meshgrid(X1, X2)
    Z = np.c_[X1.flatten(), X2.flatten()]
    Z = X_transform(Z) if X_transform is not None else Z
    Z = model.predict(Z).reshape(X1.shape)

    x_plt = np.linspace(X1.min(), X1.max(), 2)
    y_plt = -(model.intercept_[0] + model.coef_[0, 0] * x_plt) / model.coef_[0, 1]
    plt.contourf(X1, X2, Z, levels=2, cmap=RdGn, alpha=0.05)
    if show_line:
        plt.plot(x_plt, y_plt, color='purple', label='Разделяющяя прямая')
    plt.title(title)
    plt.legend()

Изучим логистическую регрессию на примере синтетических данных.

Сгенерируем выборку размера $N = 1024$ из двухмерного нормального распределения $$X_1, \ldots, X_N \sim \Normal(0, I^{2x2}).$$

В качестве целевой переменной рассмотрим $$ y = (y_1, \ldots, y_N), \quad y_k = [2 x_{k1} - 3 x_{k2} + \eps_k > 0], \quad \eps_k \sim \Normal(0, 1a).$$

In [ ]:
N = 1024
d = 2

np.random.seed(42)

X = np.random.normal(size=(N, d))
y = (2 * X[:, 0] - 3 * X[:, 1] + np.random.normal(scale=1, size=N) > 1).astype(int)

Построим график:

In [ ]:
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=RdGn)
plt.show()

# 1. Классическая логистическая регрессия

In [ ]:
# учим классическую логистическую регрессию

Посмотрим на получившиеся коэффициенты:

In [ ]:
# code

Решающее правило выглядит так: `y = (coef_ @ x + intercept_) > 0`.


__Задача:__ Выразить отсюда разделяющую прямую:

In [ ]:
b = # code
a = # code

print(f'X2 = {a:.4f} * X1 + {b: .4f}')

In [ ]:
plot_classification(X, y, model, show_line=True, title='Логистическая регрессия')

# 2. Регуляризация в логистической регрессии

Рассмотрим 2 варианта регуляризации:
* $L_1$-регуляризация: обладает свойством отбора значимых признаков;
* $L_2$-регуляризаций: сдерживает рост коэффициентов.

In [ ]:
N = 256

X, y = make_classification(n_samples=N, 
                           n_features=10, 
                           n_informative=4, 
                           n_redundant=6, 
                           n_repeated=0,
                           class_sep=1.0,
                           scale=1,
                           n_clusters_per_class=1,
                           random_state=42)
print('X shape:', X.shape)

## 2.1. $L_1$-регуляризация
Обучим логистическую регрессию с L1 регуляризацией:

In [ ]:
# учим логистической регрессию с L1

Посмотрев на коэффициенты, видим эффект отбора признаков.

In [ ]:
# смотрим коэффициенты

Построим график влияния коэффициента регуляризации на величину коэффициентов:

In [ ]:
C_list = np.logspace(-3, 3, 128)
coef_list = []

for C in C_list:
    model_l1 = # обучение L1
    coef_list.append(model_l1.coef_[0])

coef_list = np.array(coef_list)
print(coef_list.shape)

plt.figure(figsize=(12,5))
for idx in range(coef_list.shape[1]):
    plt.plot(np.log10(C_list), coef_list[:, idx], label=f'Признак {idx}')
    
plt.xlabel('log(alpha)')
plt.ylabel('Значение коэффициента')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(np.log10(C_list), (coef_list > 0).sum(axis=1), color='blue')
plt.ylabel('Число значимых признаков')
plt.xlabel('-log(C)')
plt.show()

## 2.2. $L_2$-регуляризация

Обучим логистическую регрессию с L2 регуляризацией:

In [ ]:
# учим логистической регрессию с L2

В этот раз все коэффициенты отличны от нуля:

In [ ]:
# смотрим на коэффициенты

Построим график влияния коэффициента регуляризации на величину коэффициентов:

In [ ]:
C_list = np.logspace(-1, 4, 128)
coef_list = []

for C in C_list:
    model_l2 = LogisticRegression(penalty='l2', C=1 / C).fit(X, y)
    coef_list.append(model_l2.coef_[0])

coef_list = np.array(coef_list)
print(coef_list.shape)

plt.figure(figsize=(12,5))
for idx in range(coef_list.shape[1]):
    plt.plot(np.log10(C_list), coef_list[:, idx], label=f'Признак {idx}')
    
plt.xlabel('log(alpha)')
plt.ylabel('Значение коэффициента')
plt.legend()
plt.show()

## 3. Случай линейно неразделимых данных

Рассмотрим теперь выборку наблюдений, сосредоточенную вокруг 2 концентрических окружностей:
* окружность радиуса 1;
* окружность радиуса 0.8.

In [ ]:
N = 512

X, y = make_circles(N, random_state=42, noise=0.05, factor=0.8)

In [ ]:
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=RdGn)
plt.show()

Обучим простую логистическую регрессию:

In [ ]:
# учим простую логистическую регрессию

Смотрим коэффициенты модели:

In [ ]:
# смотрим коэффициенты

In [ ]:
plot_classification(X, y, model, show_line=True)

## 3.1. Преобразование признакового пространства

Перейдем к полярным координатам:
* $X'_1 = \arctan\left(\frac{X_1}{X_2}\right)$;
* $X'_2 = \sqrt{X_1^2 + X_2^2}$.

__Задача:__ определить функцию X_transform.

In [ ]:
def X_transform(X):
    # преобразование признаков

In [ ]:
Xt = X_transform(X)

plt.figure()
plt.scatter(Xt[:, 0], Xt[:, 1], c=y, cmap=RdGn)
plt.show()

Обучаем модель:

In [ ]:
# учим модель в полярных координатах

In [ ]:
# смотрим коэффициенты

In [ ]:
plot_classification(Xt, y, model_polar, show_line=True)

Построим график классификации для исходных признаков:

In [ ]:
plot_classification(X, y, model_polar, show_line=False, X_transform=X_transform)

# 3. Оценка качества модели. Введение


__Титаник:__
* survived -- спасен или нет;
* pclass -- социально-экономический статус (1 -- upper, 2 -- middle, 3 -- lower);
* sex -- пол;
* age -- возраст;
* sibsp -- число ближайших родственников на борту (муж/жена, брат/сестра);
* parch -- число родителей/детей на борту;
* fare -- плата за проезд;
* embarked -- место посадки.

In [ ]:
frame = sns.load_dataset('titanic')
frame = frame[['pclass', 'sex', 'age', 'fare', 'survived']].dropna()

print('Shape:', frame.shape)
frame.head()

Построим модель, которая будет предсказывать выживание человека.

In [ ]:
X = frame[['pclass', 'sex', 'age', 'fare']].copy()
y = frame['survived'].copy()

Преобразуем категориальные признаки:

In [ ]:
# категориальные признаки

Для того, чтобы проверить качество модели, разделим выборку на 2 части:
* Тренировочную часть будем использовать для обучения модели.
* Тестовую часть оставим для оценки качества модели.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.copy()
X_test = X_test.copy()
y_train = y_train.copy()
y_test = y_test.copy()

print('Train:', X_train.shape[0])
print('Test:', X_test.shape[0])

__Задача:__ Отнормировать числовые признаки.

In [ ]:
# нормируем числовые признаки

Обучим логистическую регрессию:

In [ ]:
# учим простую логистическую регрессию

In [ ]:
# предсказываем train и test

Считаем метрику __Accuracy__: `#[true == pred] / N`

__Задача:__ написать функцию, которая считает метрику accuracy.

In [ ]:
def accuracy_score(true, pred):
    return (true == pred).sum() / true.shape[0]

In [ ]:
print('Train accuracy:', accuracy_score(y_train, pred_train))
print('Test accuracy:', accuracy_score(y_test, pred_test))

## 3.1. Регуляризация и влияние на переобучение

Одним из методов борьбы с переобучением является регуляризация.

Существуют разные методы выбора регуляризации и настройки её параметров:
* метод отложенной выборки;
* метод кросс-валидации;
* байесовские методы (например, метод максимальной обоснованности).

__$L_1$-регуляризация:__

In [ ]:
C_list = np.logspace(-3, 3, 128)
coef_list = []
train_acc = []
test_acc = []

for C in C_list:
    model_l1 = # обучение L1
    
    pred_train = model_l1.predict(X_train)
    pred_test = model_l1.predict(X_test)

    train_acc.append(accuracy_score(y_train, pred_train))
    test_acc.append(accuracy_score(y_test, pred_test))
    
    coef_list.append(model_l1.coef_[0])

coef_list = np.array(coef_list)

_, [ax1, ax2] = plt.subplots(2, 1, figsize=(12,10))
for idx in range(coef_list.shape[1]):
    ax1.plot(np.log10(C_list), coef_list[:, idx], label=f'{X.columns[idx]}')
    
ax2.plot(np.log10(C_list), train_acc, color='blue', label='Train')
ax2.plot(np.log10(C_list), test_acc, color='green', label='Test')

ax1.set_title('$L_1$-регуляризация')
ax1.set_xlabel('log(alpha)')
ax2.set_xlabel('log(alpha)')
ax1.set_ylabel('Значение коэффициента')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()

plt.show()

__$L_2$-регуляризация:__

In [ ]:
C_list = np.logspace(-3, 3, 128)
coef_list = []
train_acc = []
test_acc = []

for C in C_list:
    model_l2 = # обучение L2
    
    pred_train = model_l2.predict(X_train)
    pred_test = model_l2.predict(X_test)

    train_acc.append(accuracy_score(y_train, pred_train))
    test_acc.append(accuracy_score(y_test, pred_test))
    
    coef_list.append(model_l2.coef_[0])

coef_list = np.array(coef_list)

_, [ax1, ax2] = plt.subplots(2, 1, figsize=(12,10))
for idx in range(coef_list.shape[1]):
    ax1.plot(np.log10(C_list), coef_list[:, idx], label=f'{X.columns[idx]}')
    
ax2.plot(np.log10(C_list), train_acc, color='blue', label='Train')
ax2.plot(np.log10(C_list), test_acc, color='green', label='Test')

ax1.set_title('$L_2$-регуляризация')
ax1.set_xlabel('log(alpha)')
ax2.set_xlabel('log(alpha)')
ax1.set_ylabel('Значение коэффициента')
ax2.set_ylabel('Accuracy')
ax1.legend()
ax2.legend()

plt.show()